# CS-E5710 BAYESIAN DATA ANALYSIS - PROJECT
## Table of Content:
* [1. INTRODUCTION](#1)
* [2. DESCRIPTION OF DATA AND THE ANALYSIS PROBLEM](#2)
* [3. MODELS' DESCRIPTION](#3)
* [4. MODEL COMPARSION](#4)
* [5. PREDICTIVE PERFORMANCE ASSESSMENT](#5)
* [6. DISCUSSION AND POTENTIAL IMPROVEMENTS](#6)
* [7. CONCLUSION](#7)
* [8. SELF-REFLECTION LESSONS](#8)

## 1. INTRODUCTION<a class="anchor" id="1"></a>
Predicting one's heart condition is crucial for giving proper medical decisions and possibly saving lives. The misdiagnosis of heart disease may cause serious problems, as being left untreated adequately can even lead to an irreversible damage in the heart muscle and can be life-threatening. Therefore, the correct diagnosis of the heart disease status is extremely vital not only for patients’ health but also their live. Unfortunately, the heart disease status cannot be diagnosed easily since there are many blood vessels in the human body which lead directly to the heart and it is very expensive and time consuming to check all the blood vessels’ conditions by imaging method such as coronary angiogram. Hence, before making the decision whether or not conducting complicated examination techniques, it is important for the doctor to accurately assess the patients’ heart condition based on easily measurable biometric parameters such as age, sex, chest paint types, resting blood vessels, blood cholesterol levels, etc.

The report is divided into ... different sections:

• **Section 1** introduces the application domain of the Heart Disease Detection Bayesian problem

• **Section 2** explains how the Bayesian problem is formulated. The aim of this section is to define what data points represent and which properties of a data point are used as its features and labels.

• **Section 3** visualizes the data and relation between the features and the labels
• ...

## 2. DESCRIPTION OF DATA AND THE ANALYSIS PROBLEM<a class="anchor" id="2"></a>

1. age
2. sex: 0: female, 1: male
3. cp: chest pain type (4 values) 1 = typical angina, 2 = atypical angina, 3 = non-anginal pain, 4 = asymptomatic
4. trestbps: resting blood pressure
5. chol: serum cholestoral in mg/dl
6. fbs: fasting blood sugar > 120 mg/dl (1 = True, 0 = False)
7. restecg: resting electrocardiographic results (values 0 = normal, 1 = having ST-T wave abnormality, 2 = showing proable or definite left ventricular hypertrophy by Estes's criteria)
8. thalach: maximum heart rate achieved
9. exang: exercise induced angina
10. oldpeak = ST depression induced by exercise relative to rest
11. slope: the slope of the peak exercise ST segment
12. ca: number of major vessels (0-4) colored by flourosopy
13. thal: 3 = normal; 6 = fixed defect; 7 = reversable defect
14. target: angiographic disease status (0 = <50% diameter narrowing _ no presence, 1% = >50% diameter narrowing _ presence)

In [1]:
### Normalizing numeric features value
from sklearn.preprocessing import StandardScaler

SS = StandardScaler()
col_to_scale = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
df[col_to_scale] = SS.fit_transform(df[col_to_scale])
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,0.952197,1,3,0.763956,-0.256334,1,0,0.015443,0,1.087338,0,0,1,1
1,-1.915313,1,2,-0.092738,0.072199,0,1,1.633471,0,2.122573,0,0,2,1
2,-1.474158,0,1,-0.092738,-0.816773,0,0,0.977514,0,0.310912,2,0,2,1
3,0.180175,1,1,-0.663867,-0.198357,0,1,1.239897,0,-0.206705,2,0,2,1
4,0.290464,0,0,-0.663867,2.082050,0,1,0.583939,1,-0.379244,2,0,2,1


Our application can be modelled as Bayesian problem with **data points** representing patients who have already undergone heart tests. Each data point is characterized by 13 different health parameters such as age, sex, chest pain type, resting blood pressure, etc. The **label** (quantity of interest) of a data point is the heart disease status, for which values 0 and 1 indicate no presence and presence of a heart disease, respectively.
We gathered the data points with known label values using the patients’ health data recording available from UC Irvine machine learning heart disease repository which can be accessed via the link: https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/

The above data repository contains health records data from 4 different locations such as Cleveland and Long Beach of America, Switzerland, and Hungary. However, only the Cleveland’s dataset is in a good condition and being maintained. All other datasets are in poor condition and do have a lot of missing information. Thus, we will use the Cleveland dataset which consists of 303 data points which are suitable for our study.

## 3. MODELS' DESCRIPTION<a class="anchor" id="3"></a>

### 3.1 Linear regression model

In [ ]:
%autosave 0

import inspect

import arviz as az
import nest_asyncio
import pandas as pd
from IPython.display import display, display_markdown

import linear_regression as lin

data = pd.read_csv('heart.csv')
samples = data[data.columns.difference(['target'])]
outcomes = data['target']


#### Model description

The multivariable linear regression model describes the relation of given parameters to the probability of detecting disease.

\begin{align}
    \theta_i = \alpha_1 + \beta_1x_{1i} + \alpha_2 + \beta_2x_{2i} + ... + \alpha_J + \beta_Jx_{Ji}
\end{align}

where
- $\theta_i$ is the probability of detecting disease
- $\alpha_.$ and $\beta_.$ is the intercept and scalar response parameters of $\beta{x}+\alpha$
- $J$ is the number of studied variables or number of dimensions from given data

Stan model embeds the necessary data normalization and linearly scales $x_.$ to range $[0;1]$.

Spliting $\alpha$ into $J$ parts is rather unconventional, but it allows to better visualise correlation for each parameter of interest separately.

Thus, it gives a set of separate $kx + b$ -like equations that are easy to visualise and validate based on expert knowledge.

See the related Stan model source code in [Appendix A](#Appendix-A).


In [ ]:
%%capture --no-display

nest_asyncio.apply()
model = lin.build(samples, outcomes)
fit = lin.sample(model)

Below is the visualisation of the linear correlation of disease probability and parameter values shown in red lines. Blue spots show the distribution of given data, jittered along the x-axis to improve readability. y-axis illustrates the estimated probability of disease.

TODO:2 mention that we are excluding the non-significant factors from further analysis to get better elpd values

In [ ]:
lin.plot_draws(fit, samples)


#### Convergence diagnostics

In [ ]:
import diagnostics

diagnostics.convergence(fit, var_names=['alpha', 'beta', 'sigma'])


Based on `r_hat` values, all chains have successfully converged for all estimated variables. None of them exceeds the set threshold value $1.01$.

TODO:1 interpret ess, n_eff, tree depth and divergences

#### Loo

In [ ]:
diagnostics.psis_loo_summary(fit, 'Linear')

As can be seen above, the PSIS-LOO doesn't provide reliable estimate due to a large number of Pareto-K values higher than 1 (mean K value is about 3 times higher). Due to that fact can need run K-Fold validation to get more reliable predictive accuracy estimates and provide some intuition about an actual predictive accuracy.

The source code of `k_fold_cv` function can be found in  [Appendix C](#Appendix-C).


In [ ]:
diagnostics.k_fold_cv(lin.build, lin.get_disease_prob, samples, outcomes, 5)

As can be seen, the linear model is not able to properly describe the data and give reliable estimates. It has low predictive accuracy a bit higher than 80% and produces especially high number of false positives diagnoses.

One way of improving the model could be changing its sensitivity threshold, but in best case scenario it improved the model accuracy just by a couple percents, likely just by chance. This approach is far from perfect and doesn't fix false positive predictions' error.

Hence, the linear regression is not recommended to use in real applications, but it can reveal basic insights on the underlying correlations. The next chapter consider the logistic regression model which is expected to show better results.

TODO:2 Background knowledge check if the model make sense


#### Prior sensitivity

For the prior sensitivity analysis we will use a bit more simple and much quicker version of leave-one-out validation and check within-sample predictive accuracy. This summary is quick and easy to understand but is in general an overestimate of LOO-CV because it is evaluated on the data from which the model was fit.

The PSIS-LOO values are also included in the analysis to build more solid connection between posterior log likelihood values and predictive accuracy.

For the `loo_within_sample` implementation see [Appendix D](#Appendix-D).


TODO:1 Sensitivity analysis with respect to prior choices (create sensitivity table)



### 3.2 Logistic regression model

#### 3.2.1. Model theoratical/mathematic description

The multivariable logistic regression model describes the relation of given parameters to the probability of detecting disease:
$$ Y_i|X_i \sim Bernoulli(expit(\alpha + \beta_1  x_{1i} + ... + \beta_J x_{Ji}))$$ 
$$ Log Odds(Y=1|X) = log\bigg(\frac{\theta_i}{1-\theta_i}\bigg) = \alpha + \alpha + \beta_1  x_{1i} + ... + \beta_J x_{Ji}$$
$$ i = 1..N$$
where

• $\theta_i$ is the probability of detecting disease\
• $\alpha$ and $\beta_k$ is the parameters of the logistic regression model\
• J is the number of features and N is the number of data points\

#### 3.2.2. Priors choice explaination

Although the heart disease dataset was published long time ago, there is still limited amount of medical research using it and the researches study about the relation between the parameters such  ST depression induced by exercise relative to rest or maximum heart rate achieve to the heart disease status. Therefore, we think using the previous knowledge from other heart disease cases is almost imposible for use, instead we used weakly informative priors. We have tried out different priors options and checked out the accuracy of the model. At the end, we decided to choose student distribution _ student_t(2,0,10) as a default prior for alpha and normal distribution _ normal(0, 10) as the default prior for beta.

#### 3.2.3. Model stan code

Below is the stan code for logistic regression model

In [ ]:
logistic_regression = '''
data {
    int<lower=1> N;                                     // number of data points
    int<lower=1> J;                                     // number of dimensions
    matrix[N, J] X;                                     // model input data    
    int<lower=0, upper=1> y[N];                         // outcomes
}

parameters {
    real alpha;        // intercept
    vector[J] beta;    // regression coefficient
}

model {
    // Prior
    alpha ~ student_t(2, 0, 10);
    beta ~ normal(0, 1);

    // Likelihood / distribution of y
    y ~ bernoulli_logit(alpha + X * beta);
}
generated quantities {     
    real<lower=0, upper=1> probs[N];    
    vector[N] log_lik = rep_vector(0, N);    
    real tmp;

    // Calculate LOO
    for (i in 1:N)
    {
        tmp = 0;
        for (j in 1:J)
        {
            tmp += beta[j] * X[i,j];
        }                
        log_lik[i] = bernoulli_logit_lpmf(y|alpha + tmp);
        probs[i] = inv_logit(alpha + tmp); // model
    }         
}
'''

#### 3.2.4. Convergence diagnostics (Rhat, ESS), HMC specific convergence diagnostics (divergences, tree depth)
Below image shows model building result
![title](img/logistic_model.png)

As shown in the above picture

• All the parameters's $\hat{R}$ value is close to 1; therefore, the generated chains have been converged well.

• The ESS value for the model is more than 1000, this indicates that the models will have stable estimations for most applications.

TODO tree_depth, diverging


#### 3.2.5. Posterior predictive checks and what was done to improve the model

#### 3.2.6. Sensitivity analysis with respect to the prior choices
For the sensitivity check we have tried to build the model with different options for beta distribution such as uniform, normal and double exponential (Laplace) distribution. We want check that will output models (alpha and betas) are significantly changed when we vary the beta prior.
By default the **alpha** is always drawn from **student_t(2, 0, 10)** distribution. The only exception is when the beta is drawn from uniform(-inf, inf), we also need to draw alpha from the same distribution (uniform(-inf, inf)); otherwise, the model build will be failed with any other different distribution for alpha.


| beta distribution | uniform(-inf, inf) | normal(0,1) | normal(0,10) | normal(0,100) | double_exponential(0,1) |
| --- | --- | --- |--- | --- |--- |
| alpha| -0.258 |-0.233  |-0.255  |-0.252  |-0.288|
| beta[0]| 0.964 |0.930  |0.971  |0.959  |0.922|
| beta[1]| -0.348 |-0.330  |-0.352  |-0.341  |-0.310|
| beta[2]| 0.576 |0.566  |0.571  |0.571  |0.579|
| beta[3]| -0.795 |-0.768  |-0.801  |-0.791  |-0.764|
| beta[4]| -0.998 |-0.963  |-1.004|-1.012  |-0.972|

As shown in the table above, when we change the beta's prior to be drawn from multiple different distributions, the output model does not changed much, the alpha's and betas' number stay more or less the same. Therefore, we can conclude that **our logistic regression model is not sensitive to the choice of prior.**

## 4. MODEL COMPARSION<a class="anchor" id="4"></a>

## 5. PREDICTIVE PERFORMANCE ASSESSMENT<a class="anchor" id="5"></a>
TODO:
(e.g. classification accuracy) and evaluation of practical usefulness of the accuracy. If not applicable, then explanation why in this case the predictive performance is not applicable.

## 6. DISCUSSION ISSUES AND POTENTIAL IMPROVEMENTS<a class="anchor" id="6"></a>
- The linear and logistic regression models are one of the simpliest models that can be use to predict data. Using over simplify models exposes the thread that the model will fail to precisely predict the data. In future, we want to conduct more experiment 

## 7. CONCLUSION<a class="anchor" id="7"></a>
TODO: What was learn from the data analysis

## 8. SELF-REFLECTION LESSONS<a class="anchor" id="8"></a>
TODO: What group learn during project


### Appendix A

**Stan code for linear regression model**

In [ ]:
with open('linear_regression.stan', 'r') as file:
    display_markdown(f"```cpp\n{file.read()}\n```", raw=True)


### Appendix C

**K-Fold validation**

This function implements the K-Fold validation by fitting the model with n/k elements excluded from the data. Then estimated disease probability is then compared with actual `target` value for the given test sample.

In [ ]:
display_markdown(f"```python\n{inspect.getsource(lin.k_fold_cv)}\n```", raw=True)

### Appendix D

**Leave-one-out within-sample predictive accuracy**

This function relies on `probs` values computed by Stan model's `generated quantities` block.

In [ ]:
display_markdown(f"```python\n{inspect.getsource(lin.loo_within_sample)}\n```", raw=True)